In [134]:
# Import simple dependencies

import pandas as pd
import numpy as np

In [2]:
# Load file and see first 5 rows. Having seen the excel file, seeing the first 5 rows will not give much direction to what needs to be done.

df = pd.read_csv("/mnt/c/Users/user/Downloads/Technical Assessment - amalgamated_widgets_payroll.csv")
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Employee Previous Earnings,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN,AMALGAMATED WIDGETS PTY LTD,NaN,NaN,NaN,NaN,NaN,NaN
1,Employee Previous Earnings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Page 1 of 553
2,Payroll Company,NaN,AMALGAMATED WIDGETS - AMALGAMATED WIDGETS PTY LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001POWED,Dominick Powers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,For Pay End Periods between 1/07/1982 and 25/0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Things to note after looking at data:

1. "Pay frequency" seems to always be "1W01 - Weekly", however the column cannot be removed just yet as it contains employee names.
2. "Location" also seems to always be "CARCOSA", meaning the whole column is redundant.
3. The column with "Number of" contains (at all times) "Normal Hours", "Gross Taxable Total",  "Tax (Incl Adjust)", and "Net Pay". 

Thus, my idea is to create a new dataframe, with features: EmployeeID, EmployeeName, PayDate, and other important features that each pay will have differently.

In [32]:
# 1. "Pay frequency" seems to always be "1W01 - Weekly", however the column cannot be removed just yet as it contains employee names. (TRUE)

print(df.iloc[:, 1].unique())

[nan 'Dominick Powers' 'Pay Frequency' '1W01 - Weekly' 'Crystal Morin'
 'Erma Dalton' 'Kristy Nash' 'Henrietta Randall' 'Avis Browning'
 'Abigail Gaines' 'Randell Calhoun' 'Garry Kent' 'Melanie Golden'
 'Jonathan Mcconnell' 'Merrill Gamble' 'Mauro Dixon' 'Sara Ellis'
 'Phil Rush' 'Beth Lucas' 'Nola Sims' 'Larry Delaney' 'Christy Mcintyre'
 'Milo Lyons' 'Luann Moody' 'Hans Torres' 'Velma Alford' 'Imogene Delgado'
 'Irving Pace' 'Georgina Ellison' 'Corinne Bryant' 'Stefan Miles'
 'Simon Ball' 'Rosanna Conway' 'Charley Montgomery' 'Cathy Beard'
 'Ryan Todd' 'Sammy Williamson' 'Eunice Rodriguez' 'Mercedes Dean'
 'Brandie Wall' 'Ethel Carney' 'Leonel Carson' 'Luz Mendez' 'Etta Kerr'
 'Claudio George' 'Anibal Lucas' 'Charlie Figueroa' 'Bertha West'
 'Leanne Mcgowan' 'Tamara Bailey' 'Patty Lowe']


In [34]:
# 2. "Location" also seems to always be "CARCOSA", meaning the whole column is redundant. (TRUE)

print(df.iloc[:, 3].unique())


[nan 'Location' 'CARCOSA']


In [206]:
# Test if date matches regex

import re

regex_date = r'^\d{1,2}/\d{1,2}/\d{2,4}$'

for row in df.iloc[:, 0]:
    if isinstance(row, str) and re.match(regex_date, row):
        print(row)

6/12/1982
13/12/1982
20/12/1982
27/12/1982
3/01/1983
10/01/1983
17/01/1983
24/01/1983
31/01/1983
7/02/1983
14/02/1983
21/02/1983
28/02/1983
7/03/1983
14/03/1983
21/03/1983
28/03/1983
4/04/1983
11/04/1983
18/04/1983
25/04/1983
2/05/1983
9/05/1983
16/05/1983
23/05/1983
30/05/1983
16/04/1985
23/04/1985
30/04/1985
7/05/1985
5/07/1982
12/07/1982
19/07/1982
26/07/1982
2/08/1982
9/08/1982
16/08/1982
23/08/1982
30/08/1982
6/09/1982
13/09/1982
20/09/1982
27/09/1982
4/10/1982
11/10/1982
18/10/1982
25/10/1982
1/11/1982
8/11/1982
15/11/1982
22/11/1982
29/11/1982
6/12/1982
13/12/1982
20/12/1982
27/12/1982
3/01/1983
10/01/1983
17/01/1983
24/01/1983
31/01/1983
7/02/1983
14/02/1983
21/02/1983
28/02/1983
7/03/1983
14/03/1983
21/03/1983
28/03/1983
4/04/1983
11/04/1983
18/04/1983
25/04/1983
2/05/1983
9/05/1983
16/05/1983
23/05/1983
30/05/1983
6/06/1983
13/06/1983
20/06/1983
27/06/1983
4/07/1983
11/07/1983
18/07/1983
25/07/1983
5/07/1982
12/07/1982
19/07/1982
26/07/1982
2/08/1982
9/08/1982
16/08/1982
23/0

In [242]:
# Function to extract all payroll data for all employees.

import re

payroll_index = []
entries = []
regex_date = r'^\d{1,2}/\d{1,2}/\d{2,4}$'
features = []

def PayInstanceEntry(df_input):
    for idx, row in df_input.iterrows():
        if row[0] == 'Payroll Company':
            payroll_index.append(idx)   # Separates instances of pays using the appearance of "Payroll Company"
    
    for i in range(len(payroll_index) - 1):
        emp_prev_earnings = df[payroll_index[i]+1 : payroll_index[i+1]]
        emp_id = emp_prev_earnings.iloc[0,0] 
        emp_name = emp_prev_earnings.iloc[0,1] # Directly below "Payroll company" is Employee ID and to the right of it Employee Name
        
        for ep_idx in range(len(emp_prev_earnings)):
            ep_row = emp_prev_earnings.iloc[ep_idx]
            if isinstance(ep_row[0], str) and re.match(regex_date, ep_row[0]): # Finds dates in first column
                pay_date = ep_row[0]
                entry = [pay_date, emp_id, emp_name]
                while ep_idx < len(emp_prev_earnings) and pd.isna(emp_prev_earnings.iloc[ep_idx + 1, 0]):
                    entry.append([
                        emp_prev_earnings.iloc[ep_idx + 1, 5], 
                        emp_prev_earnings.iloc[ep_idx + 1, 10], 
                        emp_prev_earnings.iloc[ep_idx + 1, 11]
                    ])
                    if emp_prev_earnings.iloc[ep_idx + 1, 5] not in features:
                        features.append(emp_prev_earnings.iloc[ep_idx + 1, 5])
                    ep_idx += 1 #Iterates through the values below the date, if they are null, take the values on the right, and keep going
                entries.append(entry)

In [243]:
PayInstanceEntry(df)

In [244]:
# List of entries
entries

[['6/12/1982',
  '001POWED',
  'Dominick Powers',
  ['Normal Hours', '23', '351.87'],
  ['Gross Taxable Total', '0', '351.87'],
  ['Tax (Incl Adjust)', '0', '0'],
  ['Net Pay', '0', '351.87']],
 ['13/12/1982',
  '001POWED',
  'Dominick Powers',
  ['Normal Hours', '46', '703.74'],
  ['Gross Taxable Total', '0', '703.74'],
  ['Tax (Incl Adjust)', '0', '-80'],
  ['Net Pay', '0', '623.74'],
  ['Superannuation', '0', '66.86']],
 ['20/12/1982',
  '001POWED',
  'Dominick Powers',
  ['Normal Hours', '37.5', '573.7'],
  ['Gross Taxable Total', '0', '573.7'],
  ['Tax (Incl Adjust)', '0', '-53'],
  ['Net Pay', '0', '520.7'],
  ['Superannuation', '0', '54.5']],
 ['27/12/1982',
  '001POWED',
  'Dominick Powers',
  ['Normal Hours', '30', '458.96'],
  ['Public Holiday - worked', '15', '573.7'],
  ['Gross Taxable Total', '0', '1032.66'],
  ['Tax (Incl Adjust)', '0', '-194'],
  ['Net Pay', '0', '838.66'],
  ['Superannuation', '0', '98.1']],
 ['3/01/1983',
  '001POWED',
  'Dominick Powers',
  ['Normal H

In [126]:
# List of unique features
features

['Normal Hours',
 'Gross Taxable Total',
 'Tax (Incl Adjust)',
 'Net Pay',
 'Superannuation',
 'Public Holiday - worked',
 'Time/Half Hours',
 'Reimburse expenses',
 'Uniforms - Reducing Balance',
 'Sat Casual Ldg',
 'Other Leave - Public Holiday',
 'Annual Leave',
 'Sick Leave',
 'Other Leave - Birthday',
 'Term AL Gross',
 'Account Payment',
 '60% Sun Perm Ldg',
 'Leave Loading 17.5%',
 'Sunday Ldg',
 'Other Leave - Unpaid Leave- No accruals',
 'Long Service Leave',
 'Term LL Gross',
 'ETP - Taxable (Post June 83) Code : O',
 '   Gratuity',
 'Sat Perm Loading',
 'Term LSL Gross',
 'Sun Cas Ldg',
 'Other Leave - Jury Duty',
 'Jury Duty Subsidy',
 'Reimburse expenses- Flu Vax',
 'Other Leave - Bereavement',
 'Other Leave - Leave w/out pay- with accr',
 'uals',
 'SUPER SUPER VOL CONT $',
 "ZZZFirst Aid -hourly retail award (don't us",
 'e)',
 'First Aid- Retail award - wkly',
 'First Aid - Retail Hourly Rate',
 'Re-imburse expense - Safety Boots',
 'Other Leave - Sick Leave - No accrual

Having 51 + 3 (ID, name, date) features is just not good, it increases complexity unnecessarily, and of course the curse of dimensionality plays here. We can decide which features to include in the table. Some can be aggregated, e.g. all leaves can be on "Leave", so let's look at which ones we can.

Sure inclusions (not in order) excluding the 3:
1. Normal Hours
2. Gross Taxable Total
3. Tax
4. Net Pay
5. Superannuation


To include some clarity for the reader, we can add another column called "Other description", which will have "First Aid", "Reimburse expenses", etc, while the numerical pay for these are included in "Other".

Changes:
1. uals and e) are disregarded.
2. Leaves will include:
    - Annual Leave
    - Sick Leave
    - Other Leave - Leave w/out pay- with accr
    - Other Leave - Birthday
    - Other Leave - Bereavement
    - Other Leave - Paternity Leave- 1 wk paid
    - Other Leave - Compassionate
    - Other Leave - Workcover- lve accrued
    - Other Leave - Unpaid Leave- No accruals
    - Other Leave - Sick Leave - No accrual
3. Loading:
    - Sat Casual Ldg
    - 60% Sun Perm Ldg
    - Sat Perm Loading
    - Sun Cas Ldg
    - Leave Loading 17.5%
    - Sun ldg
4. Half hours can be added to normal hours.
5. It is arguable whether tax adjustment can be removed. If it is removed, the information of the total tax is still available and is essentially what is needed. If it is not, it is going to be a waste of space.
6. 

Final inclusions:
1. Normal Hours
2. Gross Taxable Total
3. Tax
4. Other
5. Net Pay
6. Leave
7. Loading (?)
8. Superannuation
9. Other description

Note:
I don't think a Payroll ID is needed as each pay is unique to an employee and their week of pay.

In [226]:
leave = ['Annual Leave', 'Sick Leave', 'Other Leave - Leave w/out pay- with accr', 'Other Leave - Birthday',
         'Other Leave - Bereavement', 'Other Leave - Paternity Leave- 1 wk paid', 'Other Leave - Compassionate',
         'Other Leave - Workcover- lve accrued', 'Other Leave - Unpaid Leave- No accruals', 'Other Leave - Sick Leave - No accrual', 'Other Leave - Public Holiday']

loading = ['Sat Casual Ldg', '60% Sun Perm Ldg', 'Sat Perm Loading', 'Sun Cas Ldg', 'Leave Loading 17.5%', 'Sun ldg']
core = ['Normal Hours', "Gross Taxable Total", 'Tax (Incl Adjust)', 'Net Pay', 'Superannuation']
useless = ['uals', 'e)', '   Tax Adjustment']

#It is always cheaper to collect data into a list or a list of lists or a list of dicts, and append all to a dataframe, than initializing an empty dataframe and appending row by row. 
# Therefore my function will be based off of that.

clean_entries = []
def CleanEntries(entries): #Function to make entries into a dictionary
    for ent in entries:
        entry = {'WeeklyPayDate' : np.nan, 
                'EmployeeID' : np.nan,
                'EmployeeName' : np.nan, 
                'Normal Hours' : 0, 
                'Normal Hours Amount' : 0, 
                'Leave': 0, 
                'LeaveAmount' : 0, 
                'Loading' : 0,
                'LoadingAmount' : 0,
                'Other' : 0,
                'OtherAmount' : 0,
                'Gross Taxable Total' : 0,
                'Tax (Incl Adjust)' : 0,
                'Net Pay' : 0,
                'Superannuation' : 0,
                'OtherDesc' : []} # Initialize numerical columns to 0, so that when one employee has multiple leaves or loadings, they will be added together.
        
        entry['WeeklyPayDate'] = ent[0]
        entry['EmployeeID'] = ent[1]
        entry['EmployeeName'] = ent[2]
        
        for i in range(3, len(ent)):
            if ent[i][0] in core:
                if ent[i][0] == 'Normal Hours':
                    entry['Normal Hours'] += float(ent[i][1])
                    entry['Normal Hours Amount'] += float(ent[i][2])
                else:
                    entry[ent[i][0]] += float(ent[i][2])
            
            elif ent[i][0] in leave:
                entry['Leave'] += float(ent[i][1])
                entry['LeaveAmount'] += float(ent[i][2])
                entry['OtherDesc'].append(ent[i][0])  #OtherDesc to increase awareness
            
            elif ent[i][0] in loading:
                entry['Loading'] += float(ent[i][1])
                entry['LoadingAmount'] += float(ent[i][2])
                entry['OtherDesc'].append(ent[i][0])
            
            elif ent[i][0] in useless:
                continue
            
            else:
                entry['Other'] += float(ent[i][1])
                entry['OtherAmount'] += float(ent[i][2])
                entry['OtherDesc'].append(ent[i][0])
        clean_entries.append(entry)


        


In [227]:
CleanEntries(entries)

In [228]:
# As planned

clean_entries

[{'WeeklyPayDate': '6/12/1982',
  'EmployeeID': '001POWED',
  'EmployeeName': 'Dominick Powers',
  'Normal Hours': 23.0,
  'Normal Hours Amount': 351.87,
  'Leave': 0,
  'LeaveAmount': 0,
  'Loading': 0,
  'LoadingAmount': 0,
  'Other': 0,
  'OtherAmount': 0,
  'Gross Taxable Total': 351.87,
  'Tax (Incl Adjust)': 0.0,
  'Net Pay': 351.87,
  'Superannuation': 0,
  'OtherDesc': []},
 {'WeeklyPayDate': '13/12/1982',
  'EmployeeID': '001POWED',
  'EmployeeName': 'Dominick Powers',
  'Normal Hours': 46.0,
  'Normal Hours Amount': 703.74,
  'Leave': 0,
  'LeaveAmount': 0,
  'Loading': 0,
  'LoadingAmount': 0,
  'Other': 0,
  'OtherAmount': 0,
  'Gross Taxable Total': 703.74,
  'Tax (Incl Adjust)': -80.0,
  'Net Pay': 623.74,
  'Superannuation': 66.86,
  'OtherDesc': []},
 {'WeeklyPayDate': '20/12/1982',
  'EmployeeID': '001POWED',
  'EmployeeName': 'Dominick Powers',
  'Normal Hours': 37.5,
  'Normal Hours Amount': 573.7,
  'Leave': 0,
  'LeaveAmount': 0,
  'Loading': 0,
  'LoadingAmount': 

In [229]:
clean_entries_df = pd.DataFrame(clean_entries)

In [230]:
clean_entries_df.dtypes

WeeklyPayDate           object
EmployeeID              object
EmployeeName            object
Normal Hours           float64
Normal Hours Amount    float64
Leave                  float64
LeaveAmount            float64
Loading                float64
LoadingAmount          float64
Other                  float64
OtherAmount            float64
Gross Taxable Total    float64
Tax (Incl Adjust)      float64
Net Pay                float64
Superannuation         float64
OtherDesc               object
dtype: object

In [238]:
#Though there is merit in keeping the 'WeeklyPayDate' in datetime, it may affect readability as the convention for Australians is Day Month Year,
#and the original was in the form DD/MM/YYYY. Therefore:

clean_entries_df['WeeklyPayDate'] = pd.to_datetime(clean_entries_df['WeeklyPayDate'], format='%d/%m/%Y', dayfirst=True).dt.strftime('%d/%m/%Y')

In [232]:
# Standardize column names to PascalCase.

clean_entries_df = clean_entries_df.rename(columns={'Normal Hours':'NormalHours',
                                                    'Normal Hours Amount':'NormalHoursPay',
                                                    'Leave':'LeaveHours',
                                                    'LeaveAmount':'LeavePay',
                                                    'Loading':'LoadingHours',
                                                    'LoadingAmount':'LoadingPay',
                                                    'Other':'OtherHours',
                                                    'OtherAmount': 'OtherPay',
                                                    'Gross Taxable Total':'GrossTaxableTotal',
                                                    'Tax (Incl Adjust)':'AdjustedTax',
                                                    'Net Pay':'NetPay'})

In [239]:
# Round all numerical columns to 2 decimal places.

for col in clean_entries_df.columns:
    if clean_entries_df[col].dtype == 'float64':
        clean_entries_df[col] = clean_entries_df[col].round(2)

In [246]:
# Export as csv.
clean_entries_df.to_csv("/mnt/c/Users/user/TechAssessmentOutput.csv", index=False)
